# __SSD Demo__

In [ ]:
from detect import *
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy
import warnings
warnings.filterwarnings('ignore')

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## __Load Model__

In [3]:
# Load model checkpoint
checkpoint = './data/BEST_checkpoint_ssd300.pth.tar'
checkpoint = torch.load(checkpoint)
start_epoch = checkpoint['epoch'] + 1
best_loss = checkpoint['best_loss']
print('\nLoaded checkpoint from epoch %d. Best loss so far is %.3f.\n' % (start_epoch, best_loss))
model = checkpoint['model']
model = model.to(device)
model.eval()


Loaded checkpoint from epoch 67. Best loss so far is 2.775.



SSD300(
  (base): VGGBase(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), 

## __Object Detection on Video__

In [ ]:
vidObj = cv2.VideoCapture('./Video_experiments/SSD_Video_Test.mp4')
out = cv2.VideoWriter('./Video_experiments/SSD_Video_Detection.avi', cv2.VideoWriter_fourcc(*"ffds"), 30, (640, 360))
from time import time
count = 0
time1 = time()
while True:
    success, frame = vidObj.read()
    if success == True:
        frame = cv2.resize(frame, (300, 300), interpolation=cv2.INTER_CUBIC) #Read frame and resize
        frame1 = Image.fromarray(frame) # Convert numpy frame to PIL
        imgg = detect(model, frame1, min_score=0.2, max_overlap=0.5, top_k=200)
        imgg = numpy.array(imgg.getdata()).reshape(imgg.size[0], imgg.size[1], 3)
        frame = imgg.astype('uint8')
        frame = cv2.resize(frame, dsize=(640, 360), interpolation=cv2.INTER_CUBIC)
        out.write(frame)
        count+=1
        if(count%50 == 0):
            print('FPS ' + str(count/(time() - time1)))
    else:
        break

out.release()
vidObj.release()